In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

from TCGAMultiOmics.multiomics import MultiOmicsData
from moge.network.heterogeneous_network import HeterogeneousNetwork

from moge.visualization.plot_data import matrix_heatmap

Using TensorFlow backend.


In [4]:
tf.enable_eager_execution()

In [8]:
import pickle

# READ
with open('moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.train.pickle', 'rb') as file:
    network = pickle.load(file)

In [9]:
from moge.network.triplet_generator import OnlineTripletGenerator

generator_train = OnlineTripletGenerator(network, batch_size=256)

Using SampledDataGenerator
Ed_count: 596408 , Eu_count: 151513 , En_count: 755561


/home/jonny/PycharmProjects/MultiOmicsGraphEmbedding/moge/network/edge_generator.py:63: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  self.process_negative_sampling_edges()


Ens_count: 1789224
word index: {'A': 1, 'T': 2, 'G': 3, 'C': 4}
# of nodes to sample from (non-zero degree): 28597


In [8]:
from moge.embedding.siamese_triplet_online_embedding import SiameseOnlineTripletGraphEmbedding, OnlineTripletLoss

siamese = SiameseOnlineTripletGraphEmbedding(verbose=True)
siamese.build_keras_model(multi_gpu=False)

labels_directed SparseTensor(indices=Tensor("labels_directed/indices:0", shape=(?, 2), dtype=int64, device=/device:GPU:0), values=Tensor("labels_directed/values:0", shape=(?,), dtype=float32, device=/device:GPU:0), dense_shape=Tensor("labels_directed/shape:0", shape=(2,), dtype=int64, device=/device:GPU:0))
labels_undirected SparseTensor(indices=Tensor("labels_undirected/indices:0", shape=(?, 2), dtype=int64, device=/device:GPU:0), values=Tensor("labels_undirected/values:0", shape=(?,), dtype=float32, device=/device:GPU:0), dense_shape=Tensor("labels_undirected/shape:0", shape=(2,), dtype=int64, device=/device:GPU:0))
Embedding Tensor("embedding_1/embedding_lookup/Identity:0", shape=(?, ?, 4), dtype=float32, device=/device:GPU:0)
conv2D Tensor("lstm_lambda_2/Squeeze:0", shape=(?, ?, 192), dtype=float32, device=/device:GPU:0)
max pooling_1 Tensor("max_pooling1d_1/Squeeze:0", shape=(?, ?, 192), dtype=float32, device=/device:GPU:0)
conv1d_2 Tensor("lstm_conv_2/Relu:0", shape=(?, ?, 192), 

In [19]:
X, y = generator_train.__getitem__(1)

/home/jonny/PycharmProjects/MultiOmicsGraphEmbedding/moge/network/triplet_generator.py:121: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  adj[neg_rows[sample_indices], neg_cols[sample_indices]] = EPSILON


In [20]:
X["labels_directed"]

(matrix([[  2, 101],
         [  2, 110],
         [  3,  19],
         [  3,  77],
         [  3, 166],
         [  6,  24],
         [  7, 199],
         [  8, 160],
         [  9,  45],
         [  9, 198],
         [ 11, 254],
         [ 13,  13],
         [ 14, 222],
         [ 14, 235],
         [ 15,  50],
         [ 15, 160],
         [ 15, 238],
         [ 18,   3],
         [ 20, 114],
         [ 21,  58],
         [ 21, 169],
         [ 32,   0],
         [ 34,  75],
         [ 35, 130],
         [ 38, 235],
         [ 39, 222],
         [ 41,  46],
         [ 41,  91],
         [ 41, 136],
         [ 41, 183],
         [ 43, 213],
         [ 43, 226],
         [ 45,  74],
         [ 48, 226],
         [ 52, 195],
         [ 57,  30],
         [ 57, 210],
         [ 59, 124],
         [ 60, 191],
         [ 63,  37],
         [ 63,  61],
         [ 63,  83],
         [ 63, 176],
         [ 65, 213],
         [ 68,  80],
         [ 71, 137],
         [ 71, 184],
         [ 72

In [76]:
a = tf.SparseTensor(*X["labels_directed"])

In [77]:
tf.sparse_tensor_to_dense( a, default_value = False )

<tf.Tensor: id=131, shape=(50, 50), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [81]:
e = sparse_greater_than(a, c=0.5)
e

In [82]:
e.indices

<tf.Tensor: id=141, shape=(7, 2), dtype=int64, numpy=
array([[ 8, 16],
       [14, 39],
       [18, 47],
       [33,  8],
       [33, 16],
       [33, 41],
       [33, 48]])>

In [83]:
tf.sparse_tensor_to_dense( e, default_value = False )

<tf.Tensor: id=149, shape=(50, 50), dtype=bool, numpy=
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

In [80]:
def sparse_greater_than(s, c=0.5):
    a = tf.sparse_retain(s, tf.greater(s.values, c))
    return tf.SparseTensor(a.indices, 
                           tf.ones_like(a.values, dtype=tf.bool), 
                           s.dense_shape)

def sparse_not_equal(a, b):
    neg_b = tf.SparseTensor( b.indices, -b.values, dense_shape = b.dense_shape )
    difference = tf.sparse_add( a, neg_b )
    return sparse_not_equal_to_constant( difference, 0.0 )

# # test data
# a = tf.SparseTensor( [ [ 0, 0 ], [ 1, 4 ], [ 2, 3 ] ], [ 5.0, 6, 7 ], 
#                     dense_shape = ( 5, 5 ) )
# b = tf.SparseTensor( [ [ 0, 0 ], [ 0, 2 ], [ 2, 3 ] ], [ 5.0, 6, 2 ], 
#                     dense_shape = ( 5, 5 ) )
# e = sparse_not_equal( a, b )
# f = tf.sparse_tensor_to_dense( e, default_value = False )
# f

In [ ]:
tf.random.learned_unigram_candidate_sampler()

In [ ]:
tf.nn.learned_unigram_candidate_sampler(true_classes=,
    num_true=,
    num_sampled=1,
    unique=False,
    range_max=)